In [1]:
%tensorflow_version 1.x
!git clone https://github.com/tensorflow/cleverhans.git
!pip install cleverhans/
!pip install adversarial-robustness-toolbox

TensorFlow 1.x selected.
Cloning into 'cleverhans'...
remote: Enumerating objects: 13501, done.
remote: Total 13501 (delta 0), reused 0 (delta 0), pack-reused 13501
Receiving objects: 100% (13501/13501), 8.40 MiB | 24.52 MiB/s, done.
Resolving deltas: 100% (9494/9494), done.
Processing ./cleverhans
     |████████████████████████████████| 163kB 2.7MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
  Created wheel for cleverhans: filename=cleverhans-3.0.1-cp36-none-any.whl size=253453 sha256=985d49aa08c510a114d1d433d6b5ce558dd52719e1f89873a133347257be3645
  Stored in directory: /tmp/pip-ephem-wheel-cache-1kqlv4k8/wheels/d1/6b/1d/5cf7b3ca4c0cfc7f845628b8ed46366ab5f4f56b5483e9db7f
Successfully built cleverhans
     |████████████████████████████████| 491kB 2.7MB/s 
     |████████████████████████████████| 7.1MB 46.5MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [3]:
  
from __future__ import absolute_import, division, print_function, unicode_literals
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
import numpy as np
import matplotlib.pyplot as plt
from art.attacks import FastGradientMethod
from art.classifiers import KerasClassifier
from art.utils import load_dataset
import tensorflow as tf
import tensorflow_hub as hub
tf.compat.v1.disable_eager_execution()

#  Load the CIFAR 10 dataset
(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset(str("cifar10")) # Original Dataset
print("x_train shape: " + str(x_train.shape) + "\n" + "x_train size: " + str(x_train.size) + "\n" +
      "y_train shape: " + str(y_train.shape) + "\n" + "y_train size: " + str(y_train.size) + "\n" +
      "x_test shape: " + str(x_test.shape) + "\n" + "x_test size: " + str(x_test.size) + "\n" +
      "y_test shape: " + str(y_test.shape) + "\n" + "y_test size: " + str(y_test.size) + "\n")

#  Load the victim model
hub_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"
embed = hub.KerasLayer(hub_url, input_shape=(32,32,3) )
model = tf.keras.Sequential([
    embed,
    tf.keras.layers.Activation( 'softmax' )
])
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()
victim_classifier = KerasClassifier(model=model, clip_values=(0., 1.))

#Evaluate the victim model on the benign dataset
predictions = victim_classifier.predict(x_test) # giving the classifier the x_test of the CIFAR-10 dataset.
accuracy_benign = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test) # calculates the accuracy of the predictions
print("Accuracy on original examples for victim classifier: {}%\n".format(accuracy_benign * 100))


x_train shape: (50000, 32, 32, 3)
x_train size: 153600000
y_train shape: (50000, 10)
y_train size: 500000
x_test shape: (10000, 32, 32, 3)
x_test size: 30720000
y_test shape: (10000, 10)
y_test size: 100000



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 10)                7796426   
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total params: 7,796,426
Trainable params: 0
Non-trainable params: 7,796,426
_________________________________________________________________
Accuracy on original examples for victim classifier: 94.52000000000001%



In [5]:
    # Sampled dataset to train model
x_train, y_train = x_train[:5000], y_train[:5000] # take 5000 samples for the training set
print("x_train shape: " + str(x_train.shape) + "\n" + "x_train size: " + str(x_train.size) + "\n" +
      "y_train shape: " + str(y_train.shape) + "\n" + "y_train size: " + str(y_train.size) + "\n" +
      "x_test shape: " + str(x_test.shape) + "\n" + "x_test size: " + str(x_test.size) + "\n" +
      "y_test shape: " + str(y_test.shape) + "\n" + "y_test size: " + str(y_test.size) + "\n")

# Create the model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding="same", input_shape=x_train.shape[1:]))
model.add(Activation("relu"))
model.add(Conv2D(32, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()


#Create the classifier
copycat_classifier = KerasClassifier(model=model, clip_values=(min_, max_))

x_train shape: (5000, 32, 32, 3)
x_train size: 15360000
y_train shape: (5000, 10)
y_train size: 50000
x_test shape: (10000, 32, 32, 3)
x_test size: 30720000
y_test shape: (10000, 10)
y_test size: 100000

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_8 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 32)        0         
________________________________________________

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [6]:
from art.attacks import CopycatCNN
extraction_attack = CopycatCNN(classifier=victim_classifier, batch_size_fit=128, batch_size_query=128, nb_epochs=10, nb_stolen=1000000) 
stolen_classifier = extraction_attack.extract(x=x_train, thieved_classifier=copycat_classifier)
# Step 4: Evaluate the ART classifier on benign test examples
predictions = stolen_classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Epoch 1/10
39/39 [==============================] - 25s 638ms/step - loss: 0.3060 - accuracy: 0.8997
Epoch 2/10
39/39 [==============================] - 22s 559ms/step - loss: 0.2675 - accuracy: 0.9018
Epoch 3/10
39/39 [==============================] - 22s 556ms/step - loss: 0.2499 - accuracy: 0.9053
Epoch 4/10
39/39 [==============================] - 22s 555ms/step - loss: 0.2338 - accuracy: 0.9102
Epoch 5/10
39/39 [==============================] - 22s 556ms/step - loss: 0.2217 - accuracy: 0.9133
Epoch 6/10
39/39 [==============================] - 22s 557ms/step - loss: 0.2122 - accuracy: 0.9153
Epoch 7/10
39/39 [==============================] - 22s 556ms/step - loss: 0.2015 - accuracy: 0.9211
Epoch 8/10
39/39 [==============================] - 22s 556ms/step - loss: 0.1956 - accuracy: 0.9227
Epoch 9/10
39/39 [==============================] - 22s 558ms/step - loss: 0.1867 - accuracy: 0.9260
Epoch 10/10
39/39 [==============================] - 22s 555ms/step - loss: 0.1756 - accura

In [0]:
extraction_attack = CopycatCNN(classifier=victim_classifier, batch_size_fit=128, batch_size_query=128, nb_epochs=10, nb_stolen=1000000) 
stolen_classifier = extraction_attack.extract(x=x_train[:5000], thieved_classifier=copycat_classifier)
#  Evaluate the ART classifier on benign test examples
predictions = stolen_classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Epoch 1/10
39/39 [==============================] - 20s 522ms/step - loss: 1.4165 - accuracy: 0.4960
Epoch 2/10
39/39 [==============================] - 20s 522ms/step - loss: 1.3136 - accuracy: 0.5260
Epoch 3/10
39/39 [==============================] - 20s 523ms/step - loss: 1.2367 - accuracy: 0.5599
Epoch 4/10
39/39 [==============================] - 20s 521ms/step - loss: 1.1681 - accuracy: 0.5793
Epoch 5/10
39/39 [==============================] - 20s 525ms/step - loss: 1.0765 - accuracy: 0.6194
Epoch 6/10
39/39 [==============================] - 20s 521ms/step - loss: 1.0258 - accuracy: 0.6362
Epoch 7/10
39/39 [==============================] - 20s 523ms/step - loss: 0.9591 - accuracy: 0.6591
Epoch 8/10
39/39 [==============================] - 20s 522ms/step - loss: 0.8917 - accuracy: 0.6813
Epoch 9/10
39/39 [==============================] - 20s 524ms/step - loss: 0.8365 - accuracy: 0.7053
Epoch 10/10
39/39 [==============================] - 20s 523ms/step - loss: 0.7843 - accura

In [0]:
extraction_attack = CopycatCNN(classifier=victim_classifier, batch_size_fit=128, batch_size_query=128, nb_epochs=10, nb_stolen=1000000) 
stolen_classifier = extraction_attack.extract(x=x_train[:15000], thieved_classifier=copycat_classifier)
#  Evaluate the ART classifier on benign test examples
predictions = stolen_classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Epoch 1/10
39/39 [==============================] - 20s 522ms/step - loss: 0.7196 - accuracy: 0.7484
Epoch 2/10
39/39 [==============================] - 20s 523ms/step - loss: 0.6598 - accuracy: 0.7780
Epoch 3/10
39/39 [==============================] - 20s 522ms/step - loss: 0.6033 - accuracy: 0.7937
Epoch 4/10
39/39 [==============================] - 20s 522ms/step - loss: 0.5743 - accuracy: 0.8063
Epoch 5/10
39/39 [==============================] - 20s 522ms/step - loss: 0.5090 - accuracy: 0.8269
Epoch 6/10
39/39 [==============================] - 20s 522ms/step - loss: 0.4823 - accuracy: 0.8379
Epoch 7/10
39/39 [==============================] - 20s 521ms/step - loss: 0.4495 - accuracy: 0.8442
Epoch 8/10
39/39 [==============================] - 20s 525ms/step - loss: 0.4058 - accuracy: 0.8582
Epoch 9/10
39/39 [==============================] - 20s 522ms/step - loss: 0.3650 - accuracy: 0.8736
Epoch 10/10
39/39 [==============================] - 20s 520ms/step - loss: 0.3321 - accura

In [0]:
extraction_attack = CopycatCNN(classifier=victim_classifier, batch_size_fit=128, batch_size_query=128, nb_epochs=10, nb_stolen=2500) 
stolen_classifier = extraction_attack.extract(x=x_train[:2500], thieved_classifier=copycat_classifier)
#  Evaluate the ART classifier on benign test examples
predictions = stolen_classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Epoch 1/10
19/19 [==============================] - 10s 523ms/step - loss: 0.3065 - accuracy: 0.8890
Epoch 2/10
19/19 [==============================] - 10s 524ms/step - loss: 0.2792 - accuracy: 0.9025
Epoch 3/10
19/19 [==============================] - 10s 524ms/step - loss: 0.2309 - accuracy: 0.9215
Epoch 4/10
19/19 [==============================] - 10s 526ms/step - loss: 0.2246 - accuracy: 0.9264
Epoch 5/10
19/19 [==============================] - 10s 522ms/step - loss: 0.1960 - accuracy: 0.9387
Epoch 6/10
19/19 [==============================] - 10s 523ms/step - loss: 0.1780 - accuracy: 0.9416
Epoch 7/10
19/19 [==============================] - 10s 522ms/step - loss: 0.1736 - accuracy: 0.9428
Epoch 8/10
19/19 [==============================] - 10s 525ms/step - loss: 0.1684 - accuracy: 0.9449
Epoch 9/10
19/19 [==============================] - 10s 525ms/step - loss: 0.1462 - accuracy: 0.9494
Epoch 10/10
19/19 [==============================] - 10s 525ms/step - loss: 0.1626 - accura

In [0]:
extraction_attack = CopycatCNN(classifier=victim_classifier, batch_size_fit=128, batch_size_query=128, nb_epochs=10, nb_stolen=5000) 
stolen_classifier = extraction_attack.extract(x=x_train[:5000], thieved_classifier=copycat_classifier)
# Evaluate the ART classifier on benign test examples
predictions = stolen_classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Epoch 1/10
39/39 [==============================] - 20s 521ms/step - loss: 0.3186 - accuracy: 0.8924
Epoch 2/10
39/39 [==============================] - 20s 522ms/step - loss: 0.2514 - accuracy: 0.9195
Epoch 3/10
39/39 [==============================] - 20s 520ms/step - loss: 0.2418 - accuracy: 0.9167
Epoch 4/10
39/39 [==============================] - 20s 526ms/step - loss: 0.2235 - accuracy: 0.9215
Epoch 5/10
39/39 [==============================] - 20s 520ms/step - loss: 0.2187 - accuracy: 0.9267
Epoch 6/10
39/39 [==============================] - 20s 522ms/step - loss: 0.1853 - accuracy: 0.9375
Epoch 7/10
39/39 [==============================] - 20s 520ms/step - loss: 0.1830 - accuracy: 0.9409
Epoch 8/10
39/39 [==============================] - 20s 520ms/step - loss: 0.1924 - accuracy: 0.9299
Epoch 9/10
39/39 [==============================] - 20s 522ms/step - loss: 0.1847 - accuracy: 0.9355
Epoch 10/10
39/39 [==============================] - 20s 522ms/step - loss: 0.1547 - accura

In [0]:
extraction_attack = CopycatCNN(classifier=victim_classifier, batch_size_fit=128, batch_size_query=128, nb_epochs=10, nb_stolen=15000) 
stolen_classifier = extraction_attack.extract(x=x_train[:15000], thieved_classifier=copycat_classifier)
#  Evaluate the ART classifier on benign test examples
predictions = stolen_classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Epoch 1/10
39/39 [==============================] - 20s 520ms/step - loss: 0.1778 - accuracy: 0.9411
Epoch 2/10
39/39 [==============================] - 20s 519ms/step - loss: 0.1602 - accuracy: 0.9465
Epoch 3/10
39/39 [==============================] - 20s 522ms/step - loss: 0.1546 - accuracy: 0.9513
Epoch 4/10
39/39 [==============================] - 20s 521ms/step - loss: 0.1398 - accuracy: 0.9541
Epoch 5/10
39/39 [==============================] - 20s 521ms/step - loss: 0.1400 - accuracy: 0.9559
Epoch 6/10
39/39 [==============================] - 20s 521ms/step - loss: 0.1246 - accuracy: 0.9595
Epoch 7/10
39/39 [==============================] - 21s 526ms/step - loss: 0.1295 - accuracy: 0.9539
Epoch 8/10
39/39 [==============================] - 20s 519ms/step - loss: 0.1207 - accuracy: 0.9625
Epoch 9/10
39/39 [==============================] - 20s 521ms/step - loss: 0.1150 - accuracy: 0.9603
Epoch 10/10
39/39 [==============================] - 21s 530ms/step - loss: 0.1137 - accura